# 🔢 Advanced Practice: Arithmetic Operators in Python

This notebook digs deeper into arithmetic in Python:
- Precedence & associativity
- `/` vs `//` vs `%` (including negatives) and `divmod`
- `pow` and modular exponentiation
- Floating-point pitfalls, `math.isclose`, `math.fsum`, and exact arithmetic with `decimal`/`fractions`
- Exponentiation edge cases (e.g., `0.0 ** 0`), real vs complex roots
- Complex arithmetic with `cmath`
- Augmented assignment (`+=`) vs regular addition
- Operator overloading practice with a `Vector2` class
- Mini-exercises with assertions


## 1) Precedence & associativity
Python follows standard math-like precedence: `**` > `* / // %` > `+ -`. Exponentiation is **right-associative**.

In [1]:
print(2 + 3 * 4)        # 14 (3*4 first)
print((2 + 3) * 4)      # 20 (parentheses change order)
print(2 ** 3 ** 2)      # 2 ** (3 ** 2) = 2 ** 9 = 512
print((2 ** 3) ** 2)    # 8 ** 2 = 64

# Sanity checks
assert 2 + 3 * 4 == 14
assert 2 ** 3 ** 2 == 512
print("OK ✅ precedence & associativity")

14
20
512
64
OK ✅ precedence & associativity


## 2) `/`, `//`, `%` with positives and negatives
- `/` is true division (float)
- `//` is floor division (rounds down to `-∞`)
- `%` is the remainder with the identity: `a == (a // b) * b + (a % b)`

⚠️ With negatives, floor division rounds **down**, so results may differ from truncation.

In [2]:
pairs = [(18, 4), (-18, 4), (18, -4), (-18, -4)]
for a, b in pairs:
    print(f"a={a}, b={b}")
    print("/  ->", a / b)
    print("// ->", a // b)
    print("%  ->", a % b)
    # Identity check
    assert a == (a // b) * b + (a % b)
    print("identity holds\n")
print("OK ✅ division trio & identity")

a=18, b=4
/  -> 4.5
// -> 4
%  -> 2
identity holds

a=-18, b=4
/  -> -4.5
// -> -5
%  -> 2
identity holds

a=18, b=-4
/  -> -4.5
// -> -5
%  -> -2
identity holds

a=-18, b=-4
/  -> 4.5
// -> 4
%  -> -2
identity holds

OK ✅ division trio & identity


## 3) `divmod(a, b)` and `pow(base, exp, mod)`
- `divmod(a, b)` returns `(a // b, a % b)` efficiently
- `pow(a, b, m)` computes `(a ** b) % m` efficiently (modular exponentiation)

In [3]:
q, r = divmod(23, 5)
print(q, r)
assert (q, r) == (23 // 5, 23 % 5)

print(pow(7, 560, 561))  # Fermat-like test example; fast even for large exponents
assert pow(2, 10, 7) == (2 ** 10) % 7
print("OK ✅ divmod & modular pow")

4 3
1
OK ✅ divmod & modular pow


## 4) Floating-point realities: `isclose`, `fsum`, exact arithmetic
- Binary floats cannot exactly represent many decimals
- Use `math.isclose` for comparisons
- Use `math.fsum` for numerically stable summation
- Use `decimal.Decimal` or `fractions.Fraction` for exactness when needed (e.g., money)

In [4]:
import math
from decimal import Decimal, getcontext, ROUND_HALF_UP
from fractions import Fraction

print(0.1 + 0.2)                           # not exactly 0.3
print(math.isclose(0.1 + 0.2, 0.3))        # True with default tolerances

# fsum vs sum
xs = [1e16, 1, -1e16]
print("sum:", sum(xs))                     # 0.0 (catastrophic cancellation)
print("fsum:", math.fsum(xs))              # 1.0 (better)

# Decimal for money
getcontext().prec = 28
amount = Decimal('0.10') + Decimal('0.20')
print("Decimal:", amount)

# Fraction for exact rationals
f = Fraction(1, 10) + Fraction(2, 10)
print("Fraction:", f, "as float ->", float(f))

assert math.isclose(math.fsum(xs), 1.0)
assert amount == Decimal('0.30') and f == Fraction(3, 10)
print("OK ✅ float reality & exact arithmetic")

0.30000000000000004
True
sum: 0.0
fsum: 1.0
Decimal: 0.30
Fraction: 3/10 as float -> 0.3
OK ✅ float reality & exact arithmetic


## 5) Exponentiation edge cases
- `0.0 ** 0` is defined as `1.0` in Python
- Fractional powers of negative numbers produce complex results (or `ValueError` with `math`) — use `cmath` for complex math


In [5]:
import cmath
print(0.0 ** 0)              # 1.0
print((-4) ** 0.5)           # complex result due to fractional power

# Using cmath for clarity
print(cmath.sqrt(-4))        # 2j
z = (-1) ** (1/3)            # branch-dependent complex root
print(z)
print("abs(z)=", abs(z), "phase=", cmath.phase(z))
print("OK ✅ exponentiation edge cases")

1.0
(1.2246467991473532e-16+2j)
2j
(0.5000000000000001+0.8660254037844386j)
abs(z)= 1.0 phase= 1.0471975511965976
OK ✅ exponentiation edge cases


## 6) Complex arithmetic quick tour (`cmath`)
- Complex numbers are first-class: `a + bj`
- `abs(z)` gives magnitude, `z.real` / `z.imag` parts
- `cmath.phase`, `cmath.rect`, `cmath.polar` for polar/rectangular conversions


In [6]:
z = 3 + 4j
print(z + (1 - 2j))
print("|z|:", abs(z))
print("real, imag:", z.real, z.imag)
r, phi = cmath.polar(z)
print("polar:", (r, phi))
print("rect back:", cmath.rect(r, phi))
assert abs(abs(z) - 5.0) < 1e-12
print("OK ✅ complex arithmetic")

(4+2j)
|z|: 5.0
real, imag: 3.0 4.0
polar: (5.0, 0.9272952180016122)
rect back: (3.0000000000000004+3.9999999999999996j)
OK ✅ complex arithmetic


## 7) Augmented assignment (`+=`) vs `+`
- For immutable types (e.g., `int`, `tuple`), `+=` creates a new object
- For mutable types that implement `__iadd__`, it may modify in place
- Demonstrate with a custom numeric holder


In [7]:
class NumberBox:
    def __init__(self, value):
        self.value = value
    def __add__(self, other):
        return NumberBox(self.value + other)
    def __iadd__(self, other):
        # in-place addition
        self.value += other
        return self
    def __repr__(self):
        return f"NumberBox({self.value})"

nb1 = NumberBox(10)
nb2 = nb1 + 5
print(nb1, nb2)  # nb1 unchanged, nb2 new
nb1 += 7
print(nb1)       # modified in place
assert repr(nb1) == 'NumberBox(17)' and repr(nb2) == 'NumberBox(15)'

# Immutable example: tuple
t = (1, 2)
t_id_before = id(t)
t += (3,)
print(t, t_id_before == id(t))  # id changed (new tuple)
print("OK ✅ augmented assignment semantics")

NumberBox(10) NumberBox(15)
NumberBox(17)
(1, 2, 3) False
OK ✅ augmented assignment semantics


## 8) Operator overloading practice: `Vector2`
Implement vector arithmetic:
- `+`, `-` for vector addition/subtraction
- `*` for scalar multiply; support `scalar * vector` via `__rmul__`
- `abs(v)` for length
- dot product via `.dot(other)`
- robust equality using `math.isclose`


In [8]:
import math

class Vector2:
    __slots__ = ("x", "y")
    def __init__(self, x: float, y: float):
        self.x = float(x)
        self.y = float(y)
    def __repr__(self):
        return f"Vector2({self.x}, {self.y})"
    def __eq__(self, other):
        if not isinstance(other, Vector2):
            return NotImplemented
        return math.isclose(self.x, other.x) and math.isclose(self.y, other.y)
    def __add__(self, other):
        if not isinstance(other, Vector2):
            return NotImplemented
        return Vector2(self.x + other.x, self.y + other.y)
    def __sub__(self, other):
        if not isinstance(other, Vector2):
            return NotImplemented
        return Vector2(self.x - other.x, self.y - other.y)
    def __mul__(self, k):
        try:
            return Vector2(self.x * k, self.y * k)
        except TypeError:
            return NotImplemented
    def __rmul__(self, k):
        return self.__mul__(k)
    def __abs__(self):
        return math.hypot(self.x, self.y)
    def dot(self, other: "Vector2"):
        if not isinstance(other, Vector2):
            raise TypeError("dot expects Vector2")
        return self.x * other.x + self.y * other.y

v1 = Vector2(3, 4)
v2 = Vector2(1, -2)
print(v1 + v2, v1 - v2)
print(2 * v1, v1 * 0.5)
print("|v1|:", abs(v1))
print("dot:", v1.dot(v2))
assert abs(abs(v1) - 5.0) < 1e-12
assert (v1 + v2) == Vector2(4, 2)
assert v1.dot(v2) == 3*1 + 4*(-2)
print("OK ✅ Vector2 overloads")

Vector2(4.0, 2.0) Vector2(2.0, 6.0)
Vector2(6.0, 8.0) Vector2(1.5, 2.0)
|v1|: 5.0
dot: -5.0
OK ✅ Vector2 overloads


## 9) Performance tip: `pow` with modulus vs `**` then `%`
`pow(a, b, m)` is implemented with efficient modular exponentiation and is typically much faster for large exponents than `(a ** b) % m`.

In [9]:
import timeit
a, b, m = 123456789, 123456, 10**9+7
t1 = timeit.timeit(lambda: pow(a, b, m), number=50)
print("pow(a,b,m):", t1)

# Warning: the naive version can be *much* slower and memory-heavy; keep exponent small
b_small = 5000
t2 = timeit.timeit(lambda: (a ** b_small) % m, number=1)
print("(a**b_small)%m:", t2)
print("OK ✅ timing demo (values kept reasonable)")

pow(a,b,m): 0.00018620025366544724
(a**b_small)%m: 0.005025300197303295
OK ✅ timing demo (values kept reasonable)


## 10) Mini-exercises (assertions must pass)
1. Implement a numerically stable mean using `math.fsum`.
2. Verify `a == (a // b) * b + (a % b)` for a range of signs.
3. Write `safe_nth_root(x, n)` that returns a real root when possible, else a complex using `cmath`.
4. Extend `Vector2` with a `project_onto(other)` method that returns the projection vector of `self` onto `other`.


In [10]:
import math, cmath

# 1) stable mean
def stable_mean(seq):
    seq = list(seq)
    return math.fsum(seq) / len(seq) if seq else float('nan')

vals = [1e16, 1, -1e16]
assert math.isclose(stable_mean(vals), 1.0/3)

# 2) identity across signs
for a in range(-7, 8):
    for b in range(-5, 6):
        if b == 0:
            continue
        assert a == (a // b) * b + (a % b)

# 3) safe nth root
def safe_nth_root(x, n):
    if n == 0:
        raise ValueError("n must be non-zero")
    if x >= 0 or n % 2 == 1:
        return (x if x >= 0 else -(-x) ) ** (1/n)  # real when possible
    # otherwise complex
    return cmath.exp(cmath.log(x) / n)

assert math.isclose(safe_nth_root(27, 3), 3.0)
assert isinstance(safe_nth_root(-1, 2), complex)

# 4) add projection to Vector2
def _v2_project_onto(self, other):
    if not isinstance(other, Vector2):
        raise TypeError("other must be Vector2")
    denom = other.dot(other)
    if denom == 0:
        raise ZeroDivisionError("cannot project onto zero vector")
    k = self.dot(other) / denom
    return k * other

setattr(Vector2, 'project_onto', _v2_project_onto)
u = Vector2(3, 4)
w = Vector2(1, 0)
proj = u.project_onto(w)
assert proj == Vector2(3, 0)
print("All mini-exercises passed ✅")

All mini-exercises passed ✅
